# **Clustering for Product Matching**

## Importing packages

In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist
from difflib import SequenceMatcher, get_close_matches
import re
from tqdm import tqdm, trange
import time

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## Loading the data

In [2]:
data = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/unmatched_stockist_transactions.csv')
data.head()

,product_name,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score
0,laibuta foliar feeds,murphy foliar feed,0.68,laib,eabl,0.22
1,White Candles - Riva,white line - mint & coriander,0.49,Halar Industries Ltd,zaam industries ltd.,0.83
2,Ampiclo-dawa dry syrup,vitastar pet syrup,0.40,.,mea ltd.,0.20
3,carvedilol(vidol)6.25mg,clopidol,0.45,Cosmos ltd,cosmos ltd.,0.91
4,Amoxil 'O'125mg,mineral 1-5725,0.41,GSK,wellstock,0.31


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8614931 entries, 0 to 8614930
Data columns (total 6 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   product_name              object 
 1   best_product_match        object 
 2   product_match_score       float64
 3   manufacturer_name         object 
 4   best_manufacturer_match   object 
 5   manufacturer_match_score  float64
dtypes: float64(2), object(4)
memory usage: 394.4+ MB


In [4]:
data.isna().sum()

product_name                    1
best_product_match              0
product_match_score             0
manufacturer_name           86649
best_manufacturer_match     83377
manufacturer_match_score        0
dtype: int64

## Data Preprocessing

In [5]:
# dropping null values in product_name column
data = data.dropna(subset = ['product_name'])

# cleaning the product name column
data['product_name'] = data['product_name'].apply(lambda x: x.strip())

# changing all strings to lowercase
data = data.applymap(lambda x: x.lower() if isinstance(x, str) else x)
data.head()

,product_name,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score
0,laibuta foliar feeds,murphy foliar feed,0.68,laib,eabl,0.22
1,white candles - riva,white line - mint & coriander,0.49,halar industries ltd,zaam industries ltd.,0.83
2,ampiclo-dawa dry syrup,vitastar pet syrup,0.40,.,mea ltd.,0.20
3,carvedilol(vidol)6.25mg,clopidol,0.45,cosmos ltd,cosmos ltd.,0.91
4,amoxil 'o'125mg,mineral 1-5725,0.41,gsk,wellstock,0.31


In [6]:
data['product_name'].nunique()  

129071

In [7]:
# filtering for unique product names
unique_product_names = data['product_name'].unique()
len(unique_product_names)

129071

## Clustering

In [8]:
# vectorizing the product names
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(unique_product_names)

In [ ]:
# # choosing the k-value using the elbow method
# clusters = [1000, 2000, 3000, 4000, 5000, 6000, 7000]
# silhouette_avg = []

# for cluster in clusters:
#     kmeans = KMeans(n_clusters = cluster, random_state = 42)
#     kmeans.fit(tfidf_matrix)
#     cluster_labels = kmeans.labels_
    
#     silhouette_avg.append(silhouette_score(tfidf_matrix, cluster_labels))
    
# plt.plot(clusters, silhouette_avg, 'bx-')
# plt.xlabel('Values of K') 
# plt.ylabel('Silhouette score') 
# plt.title('Silhouette analysis For Optimal k')
# plt.show()

In [ ]:
# # clustering the vectorized data
# k_values = [400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400, 2600]
# ssd = []

# for k in k_values:
#     kmeans = KMeans(n_clusters=k, random_state=0).fit(tfidf_matrix)
#     ssd.append(kmeans.inertia_)

# # plotting the elbow curve
# plt.plot(k_values, ssd, 'bx-')
# plt.xlabel('No of clusters (k)')
# plt.ylabel('inertia')
# plt.title('The Elbow Method showing the optimal k')
# plt.show()

In [8]:
# clustering the data
num_clusters = 10000
kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(tfidf_matrix)
labels = kmeans.labels_

# creating a dataframe of the clusters
cluster_to_name = {}
for label in set(labels):
    indices = np.where(labels == label)[0]
    names = unique_product_names[indices].tolist()
    cluster_to_name[label] = names
    
# viewing the clusters
cluster_4_df = pd.DataFrame.from_dict(cluster_to_name.items())
cluster_4_df.rename(columns={0: 'cluster_id', 1: 'product_names'}, inplace=True)
cluster_4_df.set_index('cluster_id', inplace=True)
pd.set_option('display.max_colwidth', None)
cluster_4_df.head(5)

,product_names
cluster_id,
0,"[nozzle four holes, nozzle (4 holes), four holed nozzle]"
1,"[ord.nails 2"", ord.nails 3"", ord nails 5"", ord nails 3"", ord nails 2"", ord nails 4"", 3"" ord.nails 1/4, 1""ord. nails 1/4, ord. nails 2"", ord. nails 3"", ord. nails 4"", 1½'' ord. nails 1/4, 4"" ord. nails 1/4, 2"" ord. nails 1/4, 5"" ord. nails 1/4, 6"" ord. nails 1/4, ord.nails 4"", ord. nails 2 ½"", ord nails 6"", ord nails 2 1/2"", ord nails, ord nails 21/2""]"
2,"[kajamba growers, kajamba growers mash, growers kajamba]"
3,"[water heater black small, water heater black big, big black heater, water heater black]"
4,"[pliers large, pliers large]"


In [104]:
unique_names_df = pd.DataFrame({'product_name': unique_product_names,
                                'label': labels})
unique_names_df.head()
len(unique_names_df)

129071

## Topic Modeling

### Topic modeling (1)

In [120]:
# extracting most common words from each cluster in order
cluster_word_freq = {}

for doc, cluster_label in zip(unique_product_names, labels):
    words = re.split(r'\s+|-|\(|\)|/|\\|\||,', doc)
    for word in words:
        if cluster_label in cluster_word_freq:
            cluster_word_freq[cluster_label][word] = cluster_word_freq[cluster_label].get(word, 0) + 1
        else:
            cluster_word_freq[cluster_label] = {word: 1}
    
for cluster_label in cluster_word_freq:
    cluster_word_freq[cluster_label] = sorted(cluster_word_freq[cluster_label].items(), key=lambda x: x[1], reverse=True)
    
cluster_word_freq_df = pd.DataFrame.from_dict(cluster_word_freq.items())
cluster_word_freq_df.rename(columns={0: 'cluster_id', 1: 'word_freq'}, inplace=True)
cluster_word_freq_df.head()

,cluster_id,word_freq
0,5598,"[(laibuta, 7), (foliar, 7), (feed, 3), (feeds, 2), (, 2), (liquid, 1)]"
1,1253,"[(, 18), (candles, 9), (riva, 9), (white, 2), (yellow, 2), (red, 2), (black, 1), (maroon, 1), (green, 1)]"
2,8257,"[(ampiclo, 16), (dawa, 14), (, 5), (ampicilin, 3), (dry, 2), (syrup, 2), (caps, 2), (neonatal, 1), (500mg, 1), (500, 1), (suspension, 1), (250mg, 1), (5ml, 1), (capsules, 1), (0.6ml, 1), (90mg, 1), (0.6, 1)]"
3,2806,"[(vidol, 8), (carvedilol, 6), (, 5), (6.25mg, 4), (tabs, 2), (28's, 1), (6.25mgs, 1), (12.5mgs, 1)]"
4,8278,"[(amoxil, 16), (, 4), (syrup, 3), (500mg, 2), ('o'125mg, 1), (syrp, 1), (amoxycillin, 1), (60ml, 1), (cap, 1), (500, 1), (100ml, 1), (capsules, 1), (dt, 1), (caps, 1), (dispersible, 1), (suspension, 1), (original, 1), (60mls, 1)]"


In [131]:
cluster_word_freq_df['cluster_name'] = cluster_word_freq_df['word_freq'].apply(lambda x: ' '.join(word[0] for word in x[:3]))
cluster_word_freq_df

,cluster_id,word_freq,cluster_name
0,5598,"[(laibuta, 7), (foliar, 7), (feed, 3), (feeds, 2), (, 2), (liquid, 1)]",laibuta foliar feed
1,1253,"[(, 18), (candles, 9), (riva, 9), (white, 2), (yellow, 2), (red, 2), (black, 1), (maroon, 1), (green, 1)]",candles riva
2,8257,"[(ampiclo, 16), (dawa, 14), (, 5), (ampicilin, 3), (dry, 2), (syrup, 2), (caps, 2), (neonatal, 1), (500mg, 1), (500, 1), (suspension, 1), (250mg, 1), (5ml, 1), (capsules, 1), (0.6ml, 1), (90mg, 1), (0.6, 1)]",ampiclo dawa
3,2806,"[(vidol, 8), (carvedilol, 6), (, 5), (6.25mg, 4), (tabs, 2), (28's, 1), (6.25mgs, 1), (12.5mgs, 1)]",vidol carvedilol
4,8278,"[(amoxil, 16), (, 4), (syrup, 3), (500mg, 2), ('o'125mg, 1), (syrp, 1), (amoxycillin, 1), (60ml, 1), (cap, 1), (500, 1), (100ml, 1), (capsules, 1), (dt, 1), (caps, 1), (dispersible, 1), (suspension, 1), (original, 1), (60mls, 1)]",amoxil syrup
...,...,...,...
9995,1745,"[(, 2), (4.0, 1), (single, 1), (core, 1), (red, 1), (black, 1), (yellow, 1)]",4.0 single
9996,6674,"[(fuel, 2), (filter, 2), (perkins, 2), (short, 1)]",fuel filter perkins
9997,7901,"[(copem, 2), (simlaw, 1)]",copem simlaw
9998,3288,"[(bestphos, 1), (5kg, 1)]",bestphos 5kg


In [ ]:
cluster_merge = cluster_word_freq_df[['cluster_id', 'cluster_name']]
cluster_merge = cluster_merge.rename(columns={'cluster_id': 'label'})

data = data.merge(unique_names_df, on='product_name', how='left')
data = data.merge(cluster_merge, on='label', how='left')

columns = data.columns.tolist()
columns.remove('cluster_name')
columns.insert(1, 'cluster_name')
data = data[columns]

data[:50]

In [140]:
data[data['product_match_score'] > 0.75][10:20]

,product_name,cluster_name,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score,label
87455,dyclon,syrup tabs,cyclone,0.77,d.d.c.l,"addiseo,france",0.27,88
87456,arimis,jelly milking arimis,agrimos,0.77,tcl,oic ltd,0.18,2080
87457,profile,profile box long,profen,0.77,greenlife ltd,green live ltd.,0.83,2165
87458,profile,profile box long,profen,0.77,greenlife ltd,green live ltd.,0.83,2165
87459,ansal,ansal f1_tomato tomatoe,antisalm,0.77,royal seed,royal seed,0.95,831
87460,vetmin,vetmin stocklick milk,ivermin,0.77,botachem enterprises,botachem enterprises ltd.,0.87,3812
87461,nelgra,nelgra 50 50mg,integra,0.77,cosmos ltd,cosmos ltd.,0.91,1627
87462,nelgra,nelgra 50 50mg,integra,0.77,cosmos ltd,cosmos ltd.,0.91,1627
87463,premix,premix gp,premium,0.77,laibuta chemical ltd,laibuta chemicals ltd.,0.93,210
87464,egocin 10% oxytetracycline injection,egocin 10% injection,oxytetracycline 10% injection,0.77,medisel (k) ltd,medisel (k) ltd.,0.94,1768


In [135]:
# data.to_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/clustered_stockist_transactions.csv', index=False)

### Topic modeling (2)

In [141]:
# creating vectorizers for each cluster
vectorizers = []

for k in range(0, num_clusters):
    vectorizers.append(CountVectorizer(lowercase=True))
    
vectorizers[0]

CountVectorizer()

In [142]:
# vectorizing the data in each cluster
vectorized_data = []
fitted_vectorizers = []

for i, vectorizer in enumerate(vectorizers):
    indices = np.where(labels == i)[0]
    try:
        data = vectorizer.fit_transform(unique_product_names[indices])
        vectorized_data.append(data)
        fitted_vectorizers.append(vectorizer)
    except:
        print("Not enough instances in cluster {}".format(i))
        vectorized_data.append(None)

In [143]:
# getting topics per cluster
num_topics_per_cluster = 1

lda_models = []

for n in range(0, num_clusters):
    lda = LatentDirichletAllocation(n_components=num_topics_per_cluster, max_iter=10, 
                                    learning_method='online', verbose=False, random_state=0)
    lda_models.append(lda)
    
lda_models[0]

LatentDirichletAllocation(learning_method='online', n_components=1,
                          random_state=0, verbose=False)

In [144]:
# fitting the lda models
cluster_lda_data = []

for l, lda in enumerate(lda_models):
    if vectorized_data[l] != None:
        cluster_lda_data.append(lda.fit_transform(vectorized_data[l]))
        
print(f'Done fitting LDA models on {num_clusters} clusters!')

Done fitting LDA models on 10000 clusters!


In [146]:
def selected_topics(model, vectorizer, top_n=4):
    current_words = []
    keywords = []

    for idx, topic in enumerate(model.components_):
        words = [(vectorizer.get_feature_names_out()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]]
        for word in words:
            if word[0] not in current_words:
                keywords.append(word)
                current_words.append(word[0])

    keywords.sort(key=lambda x: x[1], reverse=True)
    return_values = [word[0] for word in keywords]

    return return_values


all_keywords = []

for current_vectorizer, lda in enumerate(lda_models):    
    if vectorized_data[current_vectorizer] != None:
        all_keywords.append(selected_topics(lda, fitted_vectorizers[current_vectorizer]))
        
print(f'Done extracting keywords from {num_clusters} clusters!')

Done extracting keywords from 10000 clusters!


In [147]:
keywords_df = pd.DataFrame(all_keywords)
keywords_df.fillna('', inplace=True)
keywords_df['concat_name'] = keywords_df[0] + ' ' + keywords_df[1] + ' ' + keywords_df[2] + ' ' + keywords_df[3]
keywords_df.head()

,0,1,2,3,concat_name
0,nozzle,holes,four,holed,nozzle holes four holed
1,nails,ord,1½,21,nails ord 1½ 21
2,growers,kajamba,mash,,growers kajamba mash
3,heater,black,water,big,heater black water big
4,large,pliers,,,large pliers


## Manipulating Clustered Data

In [26]:
clustered_data = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/clustered_stockist_transactions.csv')
clustered_data.head()

,product_name,cluster_name,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score,label
0,laibuta foliar feeds,laibuta foliar feed,murphy foliar feed,0.68,laib,eabl,0.22,5598
1,white candles - riva,candles riva,white line - mint & coriander,0.49,halar industries ltd,zaam industries ltd.,0.83,1253
2,ampiclo-dawa dry syrup,ampiclo dawa,vitastar pet syrup,0.40,.,mea ltd.,0.20,8257
3,carvedilol(vidol)6.25mg,vidol carvedilol,clopidol,0.45,cosmos ltd,cosmos ltd.,0.91,2806
4,amoxil 'o'125mg,amoxil syrup,mineral 1-5725,0.41,gsk,wellstock,0.31,8278


In [5]:
unique_clustered_data = clustered_data[['product_name', 'cluster_name', 'best_product_match']].drop_duplicates(subset=['product_name'], keep='first')
unique_clustered_data

,product_name,cluster_name,best_product_match
0,laibuta foliar feeds,laibuta foliar feed,murphy foliar feed
1,white candles - riva,candles riva,white line - mint & coriander
2,ampiclo-dawa dry syrup,ampiclo dawa,vitastar pet syrup
3,carvedilol(vidol)6.25mg,vidol carvedilol,clopidol
4,amoxil 'o'125mg,amoxil syrup,mineral 1-5725
...,...,...,...
8610353,lyso- spray,spray body black,limoxin-25 spray
8611023,rivacia,syrup tabs,livacare
8611109,diproson oil,diproson cream creme,dinoprostone
8611784,panga(alligator),alligator panga,aligator


In [6]:
# cleanup function
def compare(row):
    comparison = {}
    i = row['product_name']
    prods_list = row[['cluster_name', 'best_product_match']].tolist()
    if isinstance(i, str):
       comparison.update({i: get_close_matches(i, prods_list, n=1, cutoff=0.1)})
    product_name = list(comparison.keys()) if comparison else None
    match = []
    score = []
    if comparison:
       for key, value in comparison.items():
           if value:
              match.append(value[0])
              score.append(round(SequenceMatcher(None, i, value[0]).ratio(), 2))
           else:
              match.append(None)
              score.append(None)
    else:
       match.append(None)
       score.append(None)
              
    return pd.Series([match, score], index = ['match', 'score'])

In [7]:
for i in tqdm(range(10), desc="Processing", ncols=80, bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt}"):
    unique_clustered_data[['match', 'score']] = [compare(row) for _, row in unique_clustered_data.iterrows()]
    unique_clustered_data.head()

Processing: 100%|████████████████████████████████████████████████████████| 10/10


In [8]:
unique_clustered_data
dnt_unique_clustered_data = unique_clustered_data.copy()

In [9]:
unique_clustered_data['match'] = unique_clustered_data['match'].apply(lambda x: x[0])
unique_clustered_data['score'] = unique_clustered_data['score'].apply(lambda x: x[0])

unique_clustered_data

,product_name,cluster_name,best_product_match,match,score
0,laibuta foliar feeds,laibuta foliar feed,murphy foliar feed,laibuta foliar feed,0.97
1,white candles - riva,candles riva,white line - mint & coriander,candles riva,0.79
2,ampiclo-dawa dry syrup,ampiclo dawa,vitastar pet syrup,ampiclo dawa,0.69
3,carvedilol(vidol)6.25mg,vidol carvedilol,clopidol,vidol carvedilol,0.50
4,amoxil 'o'125mg,amoxil syrup,mineral 1-5725,amoxil syrup,0.50
...,...,...,...,...,...
8610353,lyso- spray,spray body black,limoxin-25 spray,limoxin-25 spray,0.67
8611023,rivacia,syrup tabs,livacare,livacare,0.67
8611109,diproson oil,diproson cream creme,dinoprostone,dinoprostone,0.67
8611784,panga(alligator),alligator panga,aligator,aligator,0.67


In [10]:
unique_clustered_data[unique_clustered_data['score'] >= 0.7].sort_values(by='score', ascending=False)

,product_name,cluster_name,best_product_match,match,score
144628,4way circular box,4way circular box,cargo box,4way circular box,1.0
274661,henke ject,henke ject,phenylject,henke ject,1.0
30980,tx 350 tuitor,tx 350 tuitor,tomato - terminator,tx 350 tuitor,1.0
946324,penikan p tube,penikan p tube,penikan,penikan p tube,1.0
895683,halisi cooking oil,halisi cooking oil,healing oil,halisi cooking oil,1.0
...,...,...,...,...,...
4051496,abanycin,syrup tabs,abamycin 10%,abamycin 10%,0.7
4051494,"2"" hinges","hinges t 3""","""hinges 3""""","""hinges 3""""",0.7
267477,finisher mash (fugo),finisher kienyeji fugo,pig finisher mash,pig finisher mash,0.7
4051257,esofag tabs,esofag kit esomeprazole,nest tabs,nest tabs,0.7


In [11]:
unique_clustered_data['go_to_match'] = np.where(unique_clustered_data['score'] >= 0.7, unique_clustered_data['match'], unique_clustered_data['cluster_name'])
unique_clustered_data

,product_name,cluster_name,best_product_match,match,score,go_to_match
0,laibuta foliar feeds,laibuta foliar feed,murphy foliar feed,laibuta foliar feed,0.97,laibuta foliar feed
1,white candles - riva,candles riva,white line - mint & coriander,candles riva,0.79,candles riva
2,ampiclo-dawa dry syrup,ampiclo dawa,vitastar pet syrup,ampiclo dawa,0.69,ampiclo dawa
3,carvedilol(vidol)6.25mg,vidol carvedilol,clopidol,vidol carvedilol,0.50,vidol carvedilol
4,amoxil 'o'125mg,amoxil syrup,mineral 1-5725,amoxil syrup,0.50,amoxil syrup
...,...,...,...,...,...,...
8610353,lyso- spray,spray body black,limoxin-25 spray,limoxin-25 spray,0.67,spray body black
8611023,rivacia,syrup tabs,livacare,livacare,0.67,syrup tabs
8611109,diproson oil,diproson cream creme,dinoprostone,dinoprostone,0.67,diproson cream creme
8611784,panga(alligator),alligator panga,aligator,aligator,0.67,alligator panga


In [12]:
unique_clustered_data

,product_name,cluster_name,best_product_match,match,score,go_to_match
0,laibuta foliar feeds,laibuta foliar feed,murphy foliar feed,laibuta foliar feed,0.97,laibuta foliar feed
1,white candles - riva,candles riva,white line - mint & coriander,candles riva,0.79,candles riva
2,ampiclo-dawa dry syrup,ampiclo dawa,vitastar pet syrup,ampiclo dawa,0.69,ampiclo dawa
3,carvedilol(vidol)6.25mg,vidol carvedilol,clopidol,vidol carvedilol,0.50,vidol carvedilol
4,amoxil 'o'125mg,amoxil syrup,mineral 1-5725,amoxil syrup,0.50,amoxil syrup
...,...,...,...,...,...,...
8610353,lyso- spray,spray body black,limoxin-25 spray,limoxin-25 spray,0.67,spray body black
8611023,rivacia,syrup tabs,livacare,livacare,0.67,syrup tabs
8611109,diproson oil,diproson cream creme,dinoprostone,dinoprostone,0.67,diproson cream creme
8611784,panga(alligator),alligator panga,aligator,aligator,0.67,alligator panga


### Cleaning matches against master list

In [13]:
# loading the master list
master_list_df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/master_list.csv')
master_list_df

,combined_name,product_name,manufacturer_name
0,aviboost aqua block; vital animal health,aviboost aqua block,vital animal health
1,aviboost cl-x blue; vital animal health,aviboost cl-x blue,vital animal health
2,aviboost nutri block; vital animal health,aviboost nutri block,vital animal health
3,aviboost spectrum; vital animal health,aviboost spectrum,vital animal health
4,aviboost poultry tonic; vital animal health,aviboost poultry tonic,vital animal health
...,...,...,...
8774,jayhawk 500sc suspension concentrate ;,jayhawk 500sc suspension concentrate,NaN
8775,cynara 72 wp wettable powder;,cynara 72 wp wettable powder,NaN
8776,twiga-amine 720sl soluble concentrate;,twiga-amine 720sl soluble concentrate,NaN
8777,ranman top 160 sc suspension concentrate;,ranman top 160 sc suspension concentrate,NaN


In [14]:
matches_cache = {}
master_list = master_list_df['product_name'].to_list()

def get_closest_match(word, possibilities: list[str]):
    word = str(word).lower()
    if found := matches_cache.get(word):
        return found

    matches = get_close_matches(word, possibilities, n=1, cutoff=0.0)
    match = matches[0] if matches else ''
    score = round(SequenceMatcher(None, word, match).ratio(), 2)
    found = {'best_match': match, 'best_score': score}
    matches_cache[word] = found

    return found         # pd.Series([word, match, score], index = ['product_name', 'match', 'score'])

found_df = unique_clustered_data['go_to_match'].apply(lambda x: get_closest_match(x, master_list))

In [15]:
len(matches_cache)

13659

In [16]:
dnt_found_df = found_df.copy()

In [17]:
found_df = found_df.apply(pd.Series)
found_df.head()

,best_match,best_score
0,murphy foliar feed,0.70
1,maxcare reviva,0.59
2,amicop 50 wp,0.56
3,vital farasi,0.48
4,raxil super 375,0.57


In [18]:
unique_clustered_data.head()
new_dnt_ucd_df = unique_clustered_data.copy()

In [19]:
unique_clustered_data = pd.concat([unique_clustered_data, found_df], axis = 1)
unique_clustered_data = unique_clustered_data.reset_index(drop=True)
unique_clustered_data = unique_clustered_data.loc[:, ~unique_clustered_data.columns.duplicated()]
unique_clustered_data.head(20)

,product_name,cluster_name,best_product_match,match,score,go_to_match,best_match,best_score
0,laibuta foliar feeds,laibuta foliar feed,murphy foliar feed,laibuta foliar feed,0.97,laibuta foliar feed,murphy foliar feed,0.70
1,white candles - riva,candles riva,white line - mint & coriander,candles riva,0.79,candles riva,maxcare reviva,0.59
2,ampiclo-dawa dry syrup,ampiclo dawa,vitastar pet syrup,ampiclo dawa,0.69,ampiclo dawa,amicop 50 wp,0.56
3,carvedilol(vidol)6.25mg,vidol carvedilol,clopidol,vidol carvedilol,0.50,vidol carvedilol,vital farasi,0.48
4,amoxil 'o'125mg,amoxil syrup,mineral 1-5725,amoxil syrup,0.50,amoxil syrup,raxil super 375,0.57
5,dryblack(n7)cheap (btty)battery,n7 battery,hyacinth beans (valore),n7 battery,0.48,n7 battery,battery cover,0.58
6,baraka fertilizer,fertilizer dap baraka,fanisi tea fertilizer,fanisi tea fertilizer,0.68,fertilizer dap baraka,frontline spray,0.56
7,omeprazole/ompra caps,omeprazole caps 20mg,beetle patch cap,omeprazole caps 20mg,0.73,omeprazole caps 20mg,soprano sc 250,0.59
8,1kg urea mea chapa,urea mea 46%n,hp supreme (can),urea mea 46%n,0.58,urea mea 46%n,furadan 4f,0.52
9,505,505 ws wh,hai 450,505 ws wh,0.50,505 ws wh,doxy-500 ws,0.50


In [20]:
unique_clustered_data = unique_clustered_data.reset_index(drop=True)
unique_clustered_data = unique_clustered_data.loc[:, ~unique_clustered_data.columns.duplicated()]
unique_clustered_data.head()

,product_name,cluster_name,best_product_match,match,score,go_to_match,best_match,best_score
0,laibuta foliar feeds,laibuta foliar feed,murphy foliar feed,laibuta foliar feed,0.97,laibuta foliar feed,murphy foliar feed,0.70
1,white candles - riva,candles riva,white line - mint & coriander,candles riva,0.79,candles riva,maxcare reviva,0.59
2,ampiclo-dawa dry syrup,ampiclo dawa,vitastar pet syrup,ampiclo dawa,0.69,ampiclo dawa,amicop 50 wp,0.56
3,carvedilol(vidol)6.25mg,vidol carvedilol,clopidol,vidol carvedilol,0.50,vidol carvedilol,vital farasi,0.48
4,amoxil 'o'125mg,amoxil syrup,mineral 1-5725,amoxil syrup,0.50,amoxil syrup,raxil super 375,0.57


In [69]:
# unique_clustered_data[(unique_clustered_data['best_score'] < 0.8) & (unique_clustered_data['best_score'] >= 0.75)].sort_values(by = 'best_score', ascending = False)[-200:-100]

In [21]:
unique_clustered_data['correct_match'] = np.where(unique_clustered_data['best_score'] >= 0.75, unique_clustered_data['best_match'], unique_clustered_data['go_to_match'])
unique_clustered_data.head()

,product_name,cluster_name,best_product_match,match,score,go_to_match,best_match,best_score,correct_match
0,laibuta foliar feeds,laibuta foliar feed,murphy foliar feed,laibuta foliar feed,0.97,laibuta foliar feed,murphy foliar feed,0.70,laibuta foliar feed
1,white candles - riva,candles riva,white line - mint & coriander,candles riva,0.79,candles riva,maxcare reviva,0.59,candles riva
2,ampiclo-dawa dry syrup,ampiclo dawa,vitastar pet syrup,ampiclo dawa,0.69,ampiclo dawa,amicop 50 wp,0.56,ampiclo dawa
3,carvedilol(vidol)6.25mg,vidol carvedilol,clopidol,vidol carvedilol,0.50,vidol carvedilol,vital farasi,0.48,vidol carvedilol
4,amoxil 'o'125mg,amoxil syrup,mineral 1-5725,amoxil syrup,0.50,amoxil syrup,raxil super 375,0.57,amoxil syrup


In [22]:
len(unique_clustered_data[unique_clustered_data['best_score'] >= 0.75])

27075

In [27]:
unique_clustered_data_merge = unique_clustered_data[['product_name', 'correct_match']]

clustered_data = clustered_data.merge(unique_clustered_data_merge, on='product_name', how='left')

columns_list = clustered_data.columns.tolist()
columns_list.remove('correct_match')
columns_list.insert(2, 'correct_match')
clustered_data = clustered_data[columns_list]

clustered_data.head()

,product_name,cluster_name,correct_match,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score,label
0,laibuta foliar feeds,laibuta foliar feed,laibuta foliar feed,murphy foliar feed,0.68,laib,eabl,0.22,5598
1,white candles - riva,candles riva,candles riva,white line - mint & coriander,0.49,halar industries ltd,zaam industries ltd.,0.83,1253
2,ampiclo-dawa dry syrup,ampiclo dawa,ampiclo dawa,vitastar pet syrup,0.40,.,mea ltd.,0.20,8257
3,carvedilol(vidol)6.25mg,vidol carvedilol,vidol carvedilol,clopidol,0.45,cosmos ltd,cosmos ltd.,0.91,2806
4,amoxil 'o'125mg,amoxil syrup,amoxil syrup,mineral 1-5725,0.41,gsk,wellstock,0.31,8278


In [24]:
# clustered_data.to_csv('final_clustered_data.csv', index = False)

## Clustering Unmatched Manufacturer Records

In [2]:
# loading the data
df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/unmatched_manufacturer_stockist_transactions.csv')
df.head()

,manufacturer_name,best_manufacturer_match,manufacturer_match_score
0,Shalina Healthcare ltd,viva healthcare ltd.,0.79
1,PI Industries LTD,zaam industries ltd.,0.79
2,Kitale Industries,kitale industries limited,0.79
3,Avivet Agro solution Ltd,agrosolutions ltd.,0.79
4,local manufacturer,"""""manufacturer",0.79


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9629125 entries, 0 to 9629124
Data columns (total 3 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   manufacturer_name         object 
 1   best_manufacturer_match   object 
 2   manufacturer_match_score  float64
dtypes: float64(1), object(2)
memory usage: 220.4+ MB


In [4]:
df.isna().sum()

manufacturer_name           4663
best_manufacturer_match        0
manufacturer_match_score       0
dtype: int64

In [5]:
df[df['manufacturer_name'].isna()]

,manufacturer_name,best_manufacturer_match,manufacturer_match_score
2277977,NaN,tononoka,0.46
2278446,NaN,tononoka,0.46
2279905,NaN,tononoka,0.46
2282419,NaN,tononoka,0.46
2284271,NaN,tononoka,0.46
...,...,...,...
7982693,NaN,tononoka,0.33
7982705,NaN,tononoka,0.33
7982710,NaN,tononoka,0.33
7982715,NaN,tononoka,0.33


### Data Preprocessing

In [6]:
# dropping null values in product_name column
df = df.dropna(subset = ['manufacturer_name'])

# cleaning the product name column
df['manufacturer_name'] = df['manufacturer_name'].apply(lambda x: x.strip())

# changing all strings to lowercase
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df.head()

,manufacturer_name,best_manufacturer_match,manufacturer_match_score
0,shalina healthcare ltd,viva healthcare ltd.,0.79
1,pi industries ltd,zaam industries ltd.,0.79
2,kitale industries,kitale industries limited,0.79
3,avivet agro solution ltd,agrosolutions ltd.,0.79
4,local manufacturer,"""""manufacturer",0.79


In [7]:
df['word_count'] = df['manufacturer_name'].apply(lambda x: len(x.strip().split()))

In [8]:
df_non_dup = df.drop_duplicates(subset='manufacturer_name', keep='first')
df_non_dup['word_count'].value_counts()

word_count
1     12361
2     10128
3      5466
4      1720
5       525
6       141
7        36
8        10
9         2
10        2
Name: count, dtype: int64

In [9]:
manufacturer_list = []

for index, row in df_non_dup.iterrows():
    word_count = row['word_count']
    manufacturer_name = row['manufacturer_name']
    
    if word_count in [1,2]:
        manufacturer_slice = manufacturer_name.strip()
        manufacturer_list.append(manufacturer_slice)
    elif word_count in [3,4,5]:
        manufacturer_slice = manufacturer_name.strip().split()[:2]
        manufacturer_list.append(' '.join(manufacturer_slice))
    else:
        manufacturer_slice = manufacturer_name.strip().split()[:3]
        manufacturer_list.append(' '.join(manufacturer_slice))
           
df_non_dup['manufacturer_slice'] = manufacturer_list        
df_non_dup.head()

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,word_count,manufacturer_slice
0,shalina healthcare ltd,viva healthcare ltd.,0.79,3,shalina healthcare
1,pi industries ltd,zaam industries ltd.,0.79,3,pi industries
2,kitale industries,kitale industries limited,0.79,2,kitale industries
3,avivet agro solution ltd,agrosolutions ltd.,0.79,4,avivet agro
4,local manufacturer,"""""manufacturer",0.79,2,local manufacturer


In [10]:
first_word = df_non_dup['manufacturer_name'].apply(lambda x: x.strip().split()[0])
first_word_un = first_word.unique()
len(first_word_un)

15932

In [11]:
unique_man_names = df_non_dup['manufacturer_slice'].unique()
len(unique_man_names)

26290

In [12]:
unique_man_names = first_word_un

### Clustering

In [13]:
# vectorizing the product names
vectorizer = TfidfVectorizer()
man_tfidf_matrix = vectorizer.fit_transform(unique_man_names)

In [29]:
# clustering the data
man_clusters = 1000
kmeans = KMeans(n_clusters=man_clusters, random_state=42).fit(man_tfidf_matrix)
man_labels = kmeans.labels_

# creating a dataframe of the clusters
man_cluster_to_name = {}
for label in set(man_labels):
    indices = np.where(man_labels == label)[0]
    # indices = np.asarray(indices, dtype=np.int32)  # Convert indices to integer array
    man_names = unique_man_names[indices].tolist()
    man_cluster_to_name[label] = man_names
    
# viewing the clusters
cluster_df_0 = pd.DataFrame.from_dict(man_cluster_to_name.items())
cluster_df_0.rename(columns={0: 'cluster_id', 1: 'product_names'}, inplace=True)
cluster_df_0.set_index('cluster_id', inplace=True)
pd.set_option('display.max_colwidth', None)
cluster_df_0.head()

,product_names
cluster_id,
0,[aliesta]
1,[bands]
2,"[fresh, la, la'fresh]"
3,"[medisel, medisel.ltd, medisel., dawa/medisel]"
4,"[start/ramg/nes/proride/crocodile, start/ramg/nes/proride, start/ramg/nes/proride/crocodile/toprich, start/ramg/nes/proride/crocodile/d&k]"


In [31]:
cluster_df_0['count'] = cluster_df_0['product_names'].apply(lambda x: len(x))
cluster_df_0[cluster_df_0['count'] > 10]

,product_names,count
cluster_id,,
24,"[shalina, pi, sigma, bestvet, crown, pl, africasia, manji, pioneer, buyline, bell, concfeed, dera, yala, lachlan, cibus, syngenta, westernseed, western, insecta, oak, pwani, medlink, cri, mazao, elgon, tata, atlas, nairobi, excel, lntracin, biopharma, delta, chongqing, barasam, p1, falcon, continental, sidai, tarajio, mola, nor, silmart, dips, lumin, hernaba, bremer, a, organix, coromadel, nerix, bukola, kapi, edge, wilux, interagro, afrego, animal, anspa, indofil, vetmed, simba, chico, agrochemicals, bharat, details, elner, truvet, safaris, r.h., murvets, bayline, cropchem, microstar, vanlo, empire, shrine, tatachemicals, shimu, mashambani, holland, boon, birds, green, allua, cropcare, utraventis, heranba, cropvet, right, micropsta, ashishi, biochem, highchem, granules, kenafric, novagen, merry, statpack, utravetis, ...]",14113
188,"[agro, agro-, vet&agro, agro-hao, agro-green, agro-sol, agro-chemist, sinoch.agro, agro-t, agro-z, synovate$agro, ken-agro]",12
357,"[vet, vet&agronomic, vet-care, vet-, afri-vet, afri@vet, modedco-vet, n.k.vet, pel-vet, avi-vet, fugo-vet, palv-vet, alf-vet, k.s.vet]",14
414,"[tri, tri-cover, tri-circle, tri-clovers, tri-, tri-ciclic, tri-clover, tri-cycle, tri-cyclic, c-tri, tri-circyle]",11
505,"[super/yog, yog, yog/nes, sal/yog, hetco/yog/class, yog,, osilot/j&l/yog, yog/s&s, rhuno/yog, crocodile/yog, combotek/yog/crocodile]",11
726,"[muchoka/field, datco/field, mpambe/w.c/field, field/emass, tr,field, t.r./field, field/, mpambe/field, bera/emass/field, field, wamutitu/field]",11


In [30]:
cluster_df_0[:50]

,product_names
cluster_id,
0,[aliesta]
1,[bands]
2,"[fresh, la, la'fresh]"
3,"[medisel, medisel.ltd, medisel., dawa/medisel]"
4,"[start/ramg/nes/proride/crocodile, start/ramg/nes/proride, start/ramg/nes/proride/crocodile/toprich, start/ramg/nes/proride/crocodile/d&k]"
5,[nanjin]
6,[vet.and]
7,[sh2-8022]
8,"[ruiru, dmbl, dmbl-ruiru, dmbl/ruiru, dmbl(ruiru)]"


In [14]:
# clustering the data
man_clusters = 3000
kmeans = KMeans(n_clusters=man_clusters, random_state=42).fit(man_tfidf_matrix)
man_labels = kmeans.labels_

# creating a dataframe of the clusters
man_cluster_to_name = {}
for label in set(man_labels):
    indices = np.where(man_labels == label)[0]
    # indices = np.asarray(indices, dtype=np.int32)  # Convert indices to integer array
    man_names = unique_man_names[indices].tolist()
    man_cluster_to_name[label] = man_names
    
# viewing the clusters
cluster_df_1 = pd.DataFrame.from_dict(man_cluster_to_name.items())
cluster_df_1.rename(columns={0: 'cluster_id', 1: 'product_names'}, inplace=True)
cluster_df_1.set_index('cluster_id', inplace=True)
pd.set_option('display.max_colwidth', None)
cluster_df_1.head()

,product_names
cluster_id,
0,[assto]
1,"[shalina, crown, pl, africasia, manji, pioneer, bell, yala, lachlan, cibus, syngenta, westernseed, insecta, oak, pwani, mazao, atlas, nairobi, lntracin, delta, chongqing, barasam, p1, continental, sidai, tarajio, mola, nor, lumin, malvi, hernaba, bremer, a, organix, coromadel, nerix, bukola, kapi, edge, wilux, interagro, animal, heranda, anspa, vetmed, chico, agrochemicals, details, truvet, safaris, r.h., murvets, bayline, cropchem, microstar, vanlo, shrine, shimu, mashambani, boon, birds, allua, cropcare, utraventis, heranba, cropvet, right, blidz, micropsta, ashishi, biochem, highchem, granules, kenafric, novagen, merry, statpack, utravetis, daima, atco, simlaw, feedo, kelchemichals, aesthetic, mutuchem, pana, cafflon, jiko, a-z, parijat, bdh, farmate, elys, kingwing, amin, grebe, agricole, jiangsu, nanjing, fixatop, ...]"
2,"[ride, pro-ride]"
3,"[z-chem, chem, vet.chem.ltd, petro-chem, z.chem, n-chem]"
4,"[reddys, dr.reddys]"


In [24]:
cluster_df_1['count'] = cluster_df_1['product_names'].apply(lambda x: len(x))
cluster_df_1[cluster_df_1['count'] > 10]

,product_names,count
cluster_id,,
1,"[shalina, crown, pl, africasia, manji, pioneer, bell, yala, lachlan, cibus, syngenta, westernseed, insecta, oak, pwani, mazao, atlas, nairobi, lntracin, delta, chongqing, barasam, p1, continental, sidai, tarajio, mola, nor, lumin, malvi, hernaba, bremer, a, organix, coromadel, nerix, bukola, kapi, edge, wilux, interagro, animal, heranda, anspa, vetmed, chico, agrochemicals, details, truvet, safaris, r.h., murvets, bayline, cropchem, microstar, vanlo, shrine, shimu, mashambani, boon, birds, allua, cropcare, utraventis, heranba, cropvet, right, blidz, micropsta, ashishi, biochem, highchem, granules, kenafric, novagen, merry, statpack, utravetis, daima, atco, simlaw, feedo, kelchemichals, aesthetic, mutuchem, pana, cafflon, jiko, a-z, parijat, bdh, farmate, elys, kingwing, amin, grebe, agricole, jiangsu, nanjing, fixatop, ...]",11997


In [ ]:
cluster_df_1[50:100]

In [10]:
# clustering the data
man_clusters = 4000
kmeans = KMeans(n_clusters=man_clusters, random_state=42).fit(man_tfidf_matrix)
man_labels = kmeans.labels_

# creating a dataframe of the clusters
man_cluster_to_name = {}
for label in set(man_labels):
    indices = np.where(man_labels == label)[0]
    man_names = unique_man_names[indices].tolist()
    man_cluster_to_name[label] = man_names
    
# viewing the clusters
cluster_df_2 = pd.DataFrame.from_dict(man_cluster_to_name.items())
cluster_df_2.rename(columns={0: 'cluster_id', 1: 'product_names'}, inplace=True)
cluster_df_2.set_index('cluster_id', inplace=True)
pd.set_option('display.max_colwidth', None)
cluster_df_2.head()

,product_names
cluster_id,
0,"[true, true vets]"
1,"[biosciences kenya ltd, biosciences kenya limited., biosciences kenya ld, biosciences kenya, biosciences kenya lmt, biosciences kenya ltd., biosciences kenya limited, biosciences ltd kenya]"
2,"[light industries ltd, light ventures industries ltd., light ventures industries limited, light ventures industries ltd, light ventures industries ltd., light venturess industries ltd, light ventures industries]"
3,"[west kenya sugar company limited, kibos sugar industries ltd, sugar, sugar company, west kenya sugar co.ltd, transmara sugar co.ltd]"
4,[vet word]


In [ ]:
cluster_df_2[:50]

In [16]:
# clustering the data
man_clusters = 10000
kmeans = KMeans(n_clusters=man_clusters, random_state=42).fit(man_tfidf_matrix)
man_labels = kmeans.labels_

# creating a dataframe of the clusters
man_cluster_to_name = {}
for label in set(man_labels):
    indices = np.where(man_labels == label)[0]
    man_names = unique_man_names[indices].tolist()
    man_cluster_to_name[label] = man_names
    
# viewing the clusters
cluster_df_3 = pd.DataFrame.from_dict(man_cluster_to_name.items())
cluster_df_3.rename(columns={0: 'cluster_id', 1: 'product_names'}, inplace=True)
cluster_df_3.set_index('cluster_id', inplace=True)
pd.set_option('display.max_colwidth', None)
cluster_df_3.head()

,product_names
cluster_id,
0,[elner]
1,"[powerex, powerex/oryx]"
2,[doshi]
3,[buparno]
4,"[tr, tr,field, cp/tr/field]"


In [ ]:
cluster_df_3[:50]

In [22]:
cluster_df_3['count'] = cluster_df_3['product_names'].apply(lambda x: len(x))
cluster_df_3[cluster_df_3['count'] > 10]

,product_names,count
cluster_id,,
23,"[sigma, pl, yala, cibus, westernseed, oak, medlink, cri, mazao, tata, biopharma, delta, barasam, continental, malvi, a, coromadel, kapi, wilux, afrego, animal, heranda, vetmed, chico, truvet, r.h., bayline, empire, shimu, boon, utraventis, micropsta, ashishi, biochem, novagen, utravetis, daima, reyoung, atco, a-z, kingwing, jojemi, agricole, fixatop, alkem, crop, boom, enterprises, amazon, asshisjblife, eagle, shangai, agrotech, sinochemical, coromandel, skant, nsanja, grain, umiraj, hecovetis, crosschem, health, shandon, dira, canon, patanisho, cussons, agrevel, joskab, coromandal, r.h, zim, farmfix, solutions, alivira, starling, balm, aesthetics, oxyplus, njonjem, packaging, licpharm, the, galpha, anhui, prime, ur, mea, botachemical, tagros, seedo, sepros, animix, pharmplan, cornix, mono, just, renta, dow, greenlife, ...]",5002


In [25]:
# clustering the data
man_clusters = 15000
kmeans = KMeans(n_clusters=man_clusters, random_state=42).fit(man_tfidf_matrix)
man_labels = kmeans.labels_

# creating a dataframe of the clusters
man_cluster_to_name = {}
for label in set(man_labels):
    indices = np.where(man_labels == label)[0]
    man_names = unique_man_names[indices].tolist()
    man_cluster_to_name[label] = man_names
    
# viewing the clusters
cluster_df_4 = pd.DataFrame.from_dict(man_cluster_to_name.items())
cluster_df_4.rename(columns={0: 'cluster_id', 1: 'product_names'}, inplace=True)
cluster_df_4.set_index('cluster_id', inplace=True)
pd.set_option('display.max_colwidth', None)
cluster_df_4.head()

,product_names
cluster_id,
0,[pyretrum]
1,"[a, r.h., a-z, r.h, b, e.a, z, t, b&d, e, i, e.u, l&a, l.g, g, k, o.c.p, u, l.g., s, n.k, j, j.k, l, e.a.b.c.l, u-, n.k., e., e.a., b.a.t, s.a.p, m., n.r.x, t., d.i, n, s,c, r, e.a.b.l, b., c&z, d.c, d-, c&k, m.m.c.s, d&k, z-c, c$k, b/s, s/a, s.a, s.c, l.c.m, i-, l。g, l,g,, n., 1, 3, h, c, j.d, o.p.i., c%c, d, m, a.g.l, p&g, a.s.p, g.e.l, j.m, j.b, a., e-, i'm, p.v.c, v.g, y.s.p, k.s, s.o.s.l, a.b.i, b.p, b&, g.i, d.a.p, u.v.l, l.a, e.e.l, l.s.e, o.l.l, i,t,o,a, l.e.d, m.s, k.d.f.f, c.a.n, m.e, j&l, u.s.b, l.c.n, h.t.c, ...]"
2,[mobedeo]
3,[haleeood]
4,"[jk, jk.]"


In [28]:
cluster_df_4['count'] = cluster_df_4['product_names'].apply(lambda x: len(x))
cluster_df_4[cluster_df_4['count'] == 1]

,product_names,count
cluster_id,,
0,[pyretrum],1
2,[mobedeo],1
3,[haleeood],1
7,[agrostar],1
9,[toprich],1
...,...,...
14995,[remedica/cyprus],1
14996,[han],1
14997,[tp-006a],1


In [ ]:
unique_man_df = pd.DataFrame({'manufacturer_name': unique_product_names,
                                'label': labels})
unique_names_df.head()
len(unique_names_df)

In [11]:
test_df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/final_clustered_data.csv')
test_df[:20]

,product_name,cluster_name,correct_match,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score,label
0,laibuta foliar feeds,laibuta foliar feed,laibuta foliar feed,murphy foliar feed,0.68,laib,eabl,0.22,5598
1,white candles - riva,candles riva,candles riva,white line - mint & coriander,0.49,halar industries ltd,zaam industries ltd.,0.83,1253
2,ampiclo-dawa dry syrup,ampiclo dawa,ampiclo dawa,vitastar pet syrup,0.40,.,mea ltd.,0.20,8257
3,carvedilol(vidol)6.25mg,vidol carvedilol,vidol carvedilol,clopidol,0.45,cosmos ltd,cosmos ltd.,0.91,2806
4,amoxil 'o'125mg,amoxil syrup,amoxil syrup,mineral 1-5725,0.41,gsk,wellstock,0.31,8278
5,dryblack(n7)cheap (btty)battery,n7 battery,n7 battery,hyacinth beans (valore),0.37,mtr/his/field,tarime suppliers ltd.,0.34,1543
6,baraka fertilizer,fertilizer dap baraka,fertilizer dap baraka,fanisi tea fertilizer,0.68,cfao kenya ltd.c,cfao kenya ltd.,0.94,286
7,omeprazole/ompra caps,omeprazole caps 20mg,omeprazole caps 20mg,beetle patch cap,0.49,@,"zzagro vietnam co.,ltd-vietnam",0.00,6785
8,1kg urea mea chapa,urea mea 46%n,urea mea 46%n,hp supreme (can),0.41,mea ltd.,mea ltd.,0.94,1521
9,505,505 ws wh,505 ws wh,hai 450,0.40,w.s,wellstock,0.31,7966


In [12]:
test_df[test_df['product_name'] == 'feeders']

,product_name,cluster_name,correct_match,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score,label
914542,feeders,feeders plastic yellow,feeders plastic yellow,weedless,0.67,local,oic ltd,0.15,8259
914724,feeders,feeders plastic yellow,feeders plastic yellow,weedless,0.67,local,oic ltd,0.15,8259
918548,feeders,feeders plastic yellow,feeders plastic yellow,weedless,0.67,local,oic ltd,0.15,8259
919651,feeders,feeders plastic yellow,feeders plastic yellow,weedless,0.67,farm feeds ltd.,farm feeds ltd.,0.97,8259
919840,feeders,feeders plastic yellow,feeders plastic yellow,weedless,0.67,kenploy,kencrop kenya ltd.,0.38,8259
...,...,...,...,...,...,...,...,...,...
8608070,feeders,feeders plastic yellow,feeders plastic yellow,weedless,0.67,exoctic,oic ltd,0.40,8259
8608127,feeders,feeders plastic yellow,feeders plastic yellow,weedless,0.67,exoctic,oic ltd,0.40,8259
8612121,feeders,feeders plastic yellow,feeders plastic yellow,weedless,0.67,kenpoly,polypipes,0.47,8259
8614005,feeders,feeders plastic yellow,feeders plastic yellow,weedless,0.67,ngombe feeds industries,sayaji industries,0.63,8259


In [13]:
test_df.shape

(8614930, 9)

In [14]:
test_df = test_df.drop_duplicates(subset='product_name', keep='first')
test_df.shape

(129071, 9)

In [15]:
test_df.to_csv('clustered_product_names.csv', index = False)

In [2]:
df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/product_names_deets.csv')
df.head()

,product_name,best_product_match,product_match_score
0,GF Chick Mash,chick mash,0.87
1,penstrep,penistrep,0.94
2,Vectoclor Plus,vectoclor plus ec,0.90
3,Vita booster,vita boost,0.91
4,Ncd 100doses,kinetic 100 ec,0.54


In [3]:
df['new_product_name'] = df['product_name'].apply(lambda x: x.strip().lower() if isinstance(x, str) else x)
df.head()

,product_name,best_product_match,product_match_score,new_product_name
0,GF Chick Mash,chick mash,0.87,gf chick mash
1,penstrep,penistrep,0.94,penstrep
2,Vectoclor Plus,vectoclor plus ec,0.90,vectoclor plus
3,Vita booster,vita boost,0.91,vita booster
4,Ncd 100doses,kinetic 100 ec,0.54,ncd 100doses


In [4]:
df = df.drop_duplicates(subset='product_name', keep='first')
df.shape

(181746, 4)

In [9]:
final_df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/final_clustered_data.csv')
final_df.head()

,product_name,cluster_name,correct_match,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score,label
0,laibuta foliar feeds,laibuta foliar feed,laibuta foliar feed,murphy foliar feed,0.68,laib,eabl,0.22,5598
1,white candles - riva,candles riva,candles riva,white line - mint & coriander,0.49,halar industries ltd,zaam industries ltd.,0.83,1253
2,ampiclo-dawa dry syrup,ampiclo dawa,ampiclo dawa,vitastar pet syrup,0.40,.,mea ltd.,0.20,8257
3,carvedilol(vidol)6.25mg,vidol carvedilol,vidol carvedilol,clopidol,0.45,cosmos ltd,cosmos ltd.,0.91,2806
4,amoxil 'o'125mg,amoxil syrup,amoxil syrup,mineral 1-5725,0.41,gsk,wellstock,0.31,8278


In [10]:
final_df = final_df.drop_duplicates(subset='product_name', keep='first')
final_df.shape

(129071, 9)

In [11]:
final_df = final_df.rename(columns={'product_name': 'new_product_name'})
final_df.head()

,new_product_name,cluster_name,correct_match,best_product_match,product_match_score,manufacturer_name,best_manufacturer_match,manufacturer_match_score,label
0,laibuta foliar feeds,laibuta foliar feed,laibuta foliar feed,murphy foliar feed,0.68,laib,eabl,0.22,5598
1,white candles - riva,candles riva,candles riva,white line - mint & coriander,0.49,halar industries ltd,zaam industries ltd.,0.83,1253
2,ampiclo-dawa dry syrup,ampiclo dawa,ampiclo dawa,vitastar pet syrup,0.40,.,mea ltd.,0.20,8257
3,carvedilol(vidol)6.25mg,vidol carvedilol,vidol carvedilol,clopidol,0.45,cosmos ltd,cosmos ltd.,0.91,2806
4,amoxil 'o'125mg,amoxil syrup,amoxil syrup,mineral 1-5725,0.41,gsk,wellstock,0.31,8278


In [12]:
df = df.merge(final_df[['new_product_name', 'correct_match']], how='left', on='new_product_name')
df.head()

,product_name,best_product_match,product_match_score,new_product_name,correct_match
0,GF Chick Mash,chick mash,0.87,gf chick mash,NaN
1,penstrep,penistrep,0.94,penstrep,NaN
2,Vectoclor Plus,vectoclor plus ec,0.90,vectoclor plus,NaN
3,Vita booster,vita boost,0.91,vita booster,NaN
4,Ncd 100doses,kinetic 100 ec,0.54,ncd 100doses,100doses ncd gumboro


In [14]:
df['correct_match'] = np.where(df['correct_match'].isna(), df['best_product_match'], df['correct_match'])
df.head()

,product_name,best_product_match,product_match_score,new_product_name,correct_match
0,GF Chick Mash,chick mash,0.87,gf chick mash,chick mash
1,penstrep,penistrep,0.94,penstrep,penistrep
2,Vectoclor Plus,vectoclor plus ec,0.90,vectoclor plus,vectoclor plus ec
3,Vita booster,vita boost,0.91,vita booster,vita boost
4,Ncd 100doses,kinetic 100 ec,0.54,ncd 100doses,100doses ncd gumboro


In [15]:
df[:20]

,product_name,best_product_match,product_match_score,new_product_name,correct_match
0,GF Chick Mash,chick mash,0.87,gf chick mash,chick mash
1,penstrep,penistrep,0.94,penstrep,penistrep
2,Vectoclor Plus,vectoclor plus ec,0.90,vectoclor plus,vectoclor plus ec
3,Vita booster,vita boost,0.91,vita booster,vita boost
4,Ncd 100doses,kinetic 100 ec,0.54,ncd 100doses,100doses ncd gumboro
5,ABZ Suspension 10ml,dabantel suspension,0.68,abz suspension 10ml,abz syrup
6,SAFARI-CIGGARETTE,sani-kit,0.48,safari-ciggarette,safari cabbage seeds
7,"1½""No.13 Bolt&Nut",endospec 300 bolus,0.46,"1½""no.13 bolt&nut",13 nut
8,Albafas 2.5% 1L,albafas 2.5%,0.89,albafas 2.5% 1l,albafas 2.5%
9,DK 777,dk777,0.91,dk 777,dk777


In [16]:
df = df[['product_name', 'new_product_name', 'correct_match']]
df.head()

,product_name,new_product_name,correct_match
0,GF Chick Mash,gf chick mash,chick mash
1,penstrep,penstrep,penistrep
2,Vectoclor Plus,vectoclor plus,vectoclor plus ec
3,Vita booster,vita booster,vita boost
4,Ncd 100doses,ncd 100doses,100doses ncd gumboro


In [17]:
df.to_csv('clean_prod_names.csv', index = False)

In [18]:
maneno_tu = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/bq-results-20230626-165305-1687798408758.csv')
maneno_tu.head()

,depot_id,revenue,avge
0,2497,30.0,30.0
1,2065,30.0,30.0
2,3079,32.0,32.0
3,1807,60.0,60.0
4,2992,180.0,90.0


In [19]:
# Calculate the 25th percentile
percentile_25 = np.percentile(maneno_tu, 25)

# Calculate the 75th percentile
percentile_75 = np.percentile(maneno_tu, 75)


In [20]:
percentile_25

2623.75

In [21]:
percentile_75

625250.0